# Importing

In [ ]:
import pandas as pd
import re 

# Data Preparation

In [ ]:
df = pd.read_csv('/kaggle/input/gsarpc3/GSARPC3.csv')
df['smiles'] = df['smiles'].str.upper()

df['target'] = df['categories'].apply(lambda x: 1 if x == "inhibitor" else 0)
print("Distribusi label:")
print(df['target'].value_counts(normalize=True))

df_train, df_test = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['target']
)


# Membuat Pattern Untuk senyawa



Hal ini dilakukan agar bisa men-tokenisasi karakter kimia seperti "Br", "Cl", "Si", "Se".

In [ ]:
pattern = re.compile(
    r"(\[[^\]]*\]|Br|Cl|Si|Se|@@?|==?|##?|/|\\|\(|\)|\.|\+|\-|%\d\d|%\d|\d|[A-Z][a-z]?|[a-z])"
)

def tokenize(smiles):
    return pattern.findall(smiles)

# Proses Tokenisasi

In [ ]:
all_tokens = []
for s in df['smiles']:
    toks = tokenize(s)
    all_tokens.extend(toks)

vocab = sorted(set(all_tokens))
token_to_int = {tok: i+1 for i, tok in enumerate(vocab)}
token_to_int['<PAD>'] = 0

vocab_size = len(token_to_int)
print("vocab_size:", vocab_size)


# Penerapan Smiles to Sequence

In [ ]:
MAX_LEN = min(250, max(len(tokenize(s)) for s in df['smiles']) + 5)

def smiles_to_seq(smiles_list, max_len):
    seqs = []
    for s in smiles_list:
        toks = tokenize(s)
        seq = [token_to_int.get(t, 0) for t in toks]
        
        if len(seq) > max_len:
            seq = seq[:max_len]
        else:
            seq += [0] * (max_len - len(seq))  
        
        seqs.append(seq)
    return np.array(seqs, dtype=np.int32)

X_train = smiles_to_seq(df_train['smiles'].tolist(), MAX_LEN)
X_test  = smiles_to_seq(df_test['smiles'].tolist(), MAX_LEN)

y_train = df_train['target'].values.astype(np.float32)
y_test  = df_test['target'].values.astype(np.float32)

print(f"X_train: {X_train.shape} | vocab: {vocab_size} | max_len: {MAX_LEN}")


Dengan metode ini, data senyawa dalam format SMILES dapat dibaca oleh model LSTM